In [1]:
import logging
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random

# === CONFIGURATION LOGGING ===
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("scraping-links-marrakech.log", encoding="utf-8"),
        logging.StreamHandler()
    ]
)

# === CONFIGURATION SELENIUM ===
path = r"C:\Users\PC\Desktop\Scrapping-chromedriver\chromedriver.exe"  # adapte ton chemin
service = Service(executable_path=path)
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
driver = webdriver.Chrome(service=service, options=options)

# === Ouvrir Google Maps ===
driver.get("https://www.google.com/maps?consent=YES")
wait = WebDriverWait(driver, 10)

# === Accepter les cookies si bouton présent ===
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[.='Tout accepter']"))
    )
    accept_button.click()
    logging.info("Cookies acceptés")
    time.sleep(2)
except:
    logging.warning("Aucun bouton de consentement détecté")

# === Recherche des restaurants ===
search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
search_box.send_keys("restaurants in marrakech, Morocco")
search_box.send_keys(Keys.ENTER)

# Attendre la sidebar
sidebar = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//div[@role='feed']"))
)

# === Scroll pour charger tous les résultats ===
while True:
    driver.execute_script("arguments[0].scrollTop += arguments[0].offsetHeight;", sidebar)
    time.sleep(random.uniform(1.5, 3.0))

    results = driver.find_elements(By.XPATH, "//a[contains(@href,'/maps/place')]")
    logging.info(f"Résultats actuels : {len(results)}")

    try:
        end_message = driver.find_element(By.CLASS_NAME, "HlvSq")
        if end_message.is_displayed():
            logging.info("Fin des résultats détectée.")
            break
    except:
        pass

# === Extraire les liens ===
restaurants_links = list(set([elem.get_attribute("href") for elem in results if elem.get_attribute("href")]))
logging.info(f"{len(restaurants_links)} restaurants détectés.")

# === Sauvegarde dans Excel ===
df_links = pd.DataFrame(restaurants_links, columns=["Lien"])
df_links["Ville"] = "marrakech"  # ajouter la ville
df_links.to_excel("restaurants_links_marrakech.xlsx", index=False)

driver.quit()
logging.info("Collecte des liens terminée. Fichier enregistré : restaurants_links.xlsx")

2025-08-26 12:12:25,898 - WARNING - Aucun bouton de consentement détecté
2025-08-26 12:12:30,174 - INFO - Résultats actuels : 5
2025-08-26 12:12:32,894 - INFO - Résultats actuels : 5
2025-08-26 12:12:35,629 - INFO - Résultats actuels : 5
2025-08-26 12:12:38,406 - INFO - Résultats actuels : 5
2025-08-26 12:12:40,786 - INFO - Résultats actuels : 5
2025-08-26 12:12:42,772 - INFO - Résultats actuels : 5
2025-08-26 12:12:44,877 - INFO - Résultats actuels : 5
2025-08-26 12:12:46,831 - INFO - Résultats actuels : 5
2025-08-26 12:12:49,794 - INFO - Résultats actuels : 5
2025-08-26 12:12:52,560 - INFO - Résultats actuels : 5
2025-08-26 12:12:55,358 - INFO - Résultats actuels : 5
2025-08-26 12:12:58,322 - INFO - Résultats actuels : 5
2025-08-26 12:13:00,613 - INFO - Résultats actuels : 5
2025-08-26 12:13:02,545 - INFO - Résultats actuels : 5
2025-08-26 12:13:04,539 - INFO - Résultats actuels : 5
2025-08-26 12:13:06,447 - INFO - Résultats actuels : 5
2025-08-26 12:13:08,317 - INFO - Résultats actu

In [2]:
import pandas as pd
import math

# Charger le fichier des liens
input_file = "restaurants_links_marrakech.xlsx"
df = pd.read_excel(input_file)

# Calculer la taille de chaque partie
n = len(df)
part_size = math.ceil(n / 3)

# Découper et sauvegarder
for i in range(3):
    start = i * part_size
    end = min((i + 1) * part_size, n)
    df_part = df.iloc[start:end]
    output_file = f"restaurants_links_marrakech_part{i+1}.xlsx"
    df_part.to_excel(output_file, index=False)
    print(f"✅ Partie {i+1} sauvegardée : {output_file} ({len(df_part)} lignes)")

✅ Partie 1 sauvegardée : restaurants_links_marrakech_part1.xlsx (41 lignes)
✅ Partie 2 sauvegardée : restaurants_links_marrakech_part2.xlsx (41 lignes)
✅ Partie 3 sauvegardée : restaurants_links_marrakech_part3.xlsx (40 lignes)


In [3]:
import re
import os
# === CONFIGURATION LOGGING ===
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("scraping-restaurants-marrakech-part_1.log", encoding="utf-8"),
        logging.StreamHandler()
    ]
)

# === CONFIGURATION SELENIUM ===
path = r"C:\Users\PC\Desktop\Scrapping-chromedriver\chromedriver.exe"
service = Service(executable_path=path)
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=service, options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})

wait = WebDriverWait(driver, 10)

# === Fichiers ===
input_file = "restaurants_links_marrakech_part1.xlsx"   # change selon la partie que tu veux scraper
output_file = "restaurants_reviews_marrakech1.xlsx"

# === Charger les liens ===
df_links = pd.read_excel(input_file)

# === Charger ou créer fichier des résultats ===
if os.path.exists(output_file):
    df = pd.read_excel(output_file)
    logging.info(f"Fichier existant trouvé avec {len(df)} lignes.")
else:
    df = pd.DataFrame(columns=["Nom du Restaurant", "Adresse", "Latitude", "Longitude", "Prix", "Nom", "Note", "Date", "Commentaire"])
    logging.info("Nouveau fichier Excel créé.")

# === Scraping ===
for i, row in df_links.iterrows():
    restaurant_url = row["Lien"]
    ville = row["Ville"]

    try:
        driver.get(restaurant_url)
        time.sleep(3)
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[.='Tout accepter']"))
            )
            accept_button.click()
            logging.info("Cookies acceptés")
            time.sleep(2)
        except:
            logging.warning("Aucun bouton de consentement détecté")
    except Exception as e:
        logging.error(f"Impossible de charger {restaurant_url} : {e}")
        continue

    # Récupération infos restaurant
    try:
        nom_restaurant = driver.find_element(By.XPATH, "//h1[contains(@class, 'DUwDvf')]").text
    except:
        nom_restaurant = "Nom inconnu"

    try:
        adresse = driver.find_element(By.XPATH, "//div[contains(@class, 'Io6YTe')]").text
    except:
        adresse = "Adresse inconnue"

    # Vérifier si déjà sauvegardé
    if ((df["Nom du Restaurant"] == nom_restaurant) & (df["Adresse"] == adresse)).any():
        logging.info(f"Déjà récupéré : {nom_restaurant} - {adresse}")
        continue

    try:
        current_url = driver.current_url
        match = re.search(r"/@([0-9\.\-]+),([0-9\.\-]+)", current_url)
        latitude, longitude = (match.group(1), match.group(2)) if match else ("", "")
    except:
        latitude, longitude = "", ""

    try:
        prix = driver.find_element(By.XPATH, "//span[contains(text(),'MAD') or contains(text(),'د.م') or contains(text(), '$')]").text
    except:
        prix = ""

    # === Ouvrir et scraper la section Avis ===
    all_reviews = []
    try:
        avis_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Avis')]")))
        avis_button.click()
        time.sleep(3)

        scrollable_div = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@jslog,'26354;mutable:true;')]")))

        # Scroll des avis
        start_time = time.time()
        previous_height = -1
        while True:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(2)
            current_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if current_height == previous_height or (time.time() - start_time) > 60:
                break
            previous_height = current_height

        review_ids_seen = set()
        avis_elements = driver.find_elements(By.XPATH, "//div[@data-review-id]")

        for avis in avis_elements:
            review_id = avis.get_attribute("data-review-id")
            if review_id in review_ids_seen:
                continue
            review_ids_seen.add(review_id)

            nom = avis.find_element(By.CLASS_NAME, "d4r55").text if avis.find_elements(By.CLASS_NAME, "d4r55") else ""
            note = avis.find_element(By.CLASS_NAME, "kvMYJc").get_attribute("aria-label") if avis.find_elements(By.CLASS_NAME, "kvMYJc") else ""
            date = avis.find_element(By.CLASS_NAME, "rsqaWe").text if avis.find_elements(By.CLASS_NAME, "rsqaWe") else ""
            commentaire = avis.find_element(By.CLASS_NAME, "wiI7pd").text if avis.find_elements(By.CLASS_NAME, "wiI7pd") else ""

            all_reviews.append([nom_restaurant, adresse, latitude, longitude, prix, nom, note, date, commentaire])

        # === Ajout au DataFrame et sauvegarde après le restaurant ===
        if all_reviews:
            df = pd.concat([df, pd.DataFrame(all_reviews, columns=df.columns)], ignore_index=True)
            df.to_excel(output_file, index=False)
            logging.info(f"Avis sauvegardés pour {nom_restaurant} ({i+1}/{len(df_links)})")
        else:
            logging.info(f"Aucun avis trouvé pour {nom_restaurant}")

    except Exception as e:
        logging.error(f"Erreur lors des avis pour {nom_restaurant} : {e}")

driver.quit()
logging.info("Scraping terminé.")

2025-08-26 12:15:12,580 - INFO - Nouveau fichier Excel créé.
2025-08-26 12:15:23,662 - WARNING - Aucun bouton de consentement détecté
2025-08-26 12:16:58,243 - INFO - Avis sauvegardés pour Chez Mmima (1/41)
2025-08-26 12:17:08,642 - WARNING - Aucun bouton de consentement détecté
2025-08-26 12:18:42,925 - INFO - Avis sauvegardés pour Las Terrazas de Andalucia (2/41)
2025-08-26 12:18:53,228 - WARNING - Aucun bouton de consentement détecté
2025-08-26 12:20:22,241 - INFO - Avis sauvegardés pour MARRAK (3/41)
2025-08-26 12:20:32,130 - WARNING - Aucun bouton de consentement détecté
2025-08-26 12:22:06,762 - INFO - Avis sauvegardés pour Safran by Kôya (4/41)
2025-08-26 12:22:16,328 - WARNING - Aucun bouton de consentement détecté
2025-08-26 12:23:50,379 - INFO - Avis sauvegardés pour Fine Mama? Restaurant & Rooftop (5/41)
2025-08-26 12:23:59,761 - WARNING - Aucun bouton de consentement détecté
2025-08-26 12:25:34,652 - INFO - Avis sauvegardés pour La Grande Brasserie by Hélène Darroze | Royal

In [4]:
import re
import os
# === CONFIGURATION LOGGING ===
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("scraping-restaurants-marrakech-part_2.log", encoding="utf-8"),
        logging.StreamHandler()
    ]
)

# === CONFIGURATION SELENIUM ===
path = r"C:\Users\PC\Desktop\Scrapping-chromedriver\chromedriver.exe"
service = Service(executable_path=path)
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=service, options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})

wait = WebDriverWait(driver, 10)

# === Fichiers ===
input_file = "restaurants_links_marrakech_part2.xlsx"   # change selon la partie que tu veux scraper
output_file = "restaurants_reviews_marrakech2.xlsx"

# === Charger les liens ===
df_links = pd.read_excel(input_file)

# === Charger ou créer fichier des résultats ===
if os.path.exists(output_file):
    df = pd.read_excel(output_file)
    logging.info(f"Fichier existant trouvé avec {len(df)} lignes.")
else:
    df = pd.DataFrame(columns=["Nom du Restaurant", "Adresse", "Latitude", "Longitude", "Prix", "Nom", "Note", "Date", "Commentaire"])
    logging.info("Nouveau fichier Excel créé.")

# === Scraping ===
for i, row in df_links.iterrows():
    restaurant_url = row["Lien"]
    ville = row["Ville"]

    try:
        driver.get(restaurant_url)
        time.sleep(3)
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[.='Tout accepter']"))
            )
            accept_button.click()
            logging.info("Cookies acceptés")
            time.sleep(2)
        except:
            logging.warning("Aucun bouton de consentement détecté")
    except Exception as e:
        logging.error(f"Impossible de charger {restaurant_url} : {e}")
        continue

    # Récupération infos restaurant
    try:
        nom_restaurant = driver.find_element(By.XPATH, "//h1[contains(@class, 'DUwDvf')]").text
    except:
        nom_restaurant = "Nom inconnu"

    try:
        adresse = driver.find_element(By.XPATH, "//div[contains(@class, 'Io6YTe')]").text
    except:
        adresse = "Adresse inconnue"

    # Vérifier si déjà sauvegardé
    if ((df["Nom du Restaurant"] == nom_restaurant) & (df["Adresse"] == adresse)).any():
        logging.info(f"Déjà récupéré : {nom_restaurant} - {adresse}")
        continue

    try:
        current_url = driver.current_url
        match = re.search(r"/@([0-9\.\-]+),([0-9\.\-]+)", current_url)
        latitude, longitude = (match.group(1), match.group(2)) if match else ("", "")
    except:
        latitude, longitude = "", ""

    try:
        prix = driver.find_element(By.XPATH, "//span[contains(text(),'MAD') or contains(text(),'د.م') or contains(text(), '$')]").text
    except:
        prix = ""

    # === Ouvrir et scraper la section Avis ===
    all_reviews = []
    try:
        avis_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Avis')]")))
        avis_button.click()
        time.sleep(3)

        scrollable_div = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@jslog,'26354;mutable:true;')]")))

        # Scroll des avis
        start_time = time.time()
        previous_height = -1
        while True:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(2)
            current_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if current_height == previous_height or (time.time() - start_time) > 60:
                break
            previous_height = current_height

        review_ids_seen = set()
        avis_elements = driver.find_elements(By.XPATH, "//div[@data-review-id]")

        for avis in avis_elements:
            review_id = avis.get_attribute("data-review-id")
            if review_id in review_ids_seen:
                continue
            review_ids_seen.add(review_id)

            nom = avis.find_element(By.CLASS_NAME, "d4r55").text if avis.find_elements(By.CLASS_NAME, "d4r55") else ""
            note = avis.find_element(By.CLASS_NAME, "kvMYJc").get_attribute("aria-label") if avis.find_elements(By.CLASS_NAME, "kvMYJc") else ""
            date = avis.find_element(By.CLASS_NAME, "rsqaWe").text if avis.find_elements(By.CLASS_NAME, "rsqaWe") else ""
            commentaire = avis.find_element(By.CLASS_NAME, "wiI7pd").text if avis.find_elements(By.CLASS_NAME, "wiI7pd") else ""

            all_reviews.append([nom_restaurant, adresse, latitude, longitude, prix, nom, note, date, commentaire])

        # === Ajout au DataFrame et sauvegarde après le restaurant ===
        if all_reviews:
            df = pd.concat([df, pd.DataFrame(all_reviews, columns=df.columns)], ignore_index=True)
            df.to_excel(output_file, index=False)
            logging.info(f"Avis sauvegardés pour {nom_restaurant} ({i+1}/{len(df_links)})")
        else:
            logging.info(f"Aucun avis trouvé pour {nom_restaurant}")

    except Exception as e:
        logging.error(f"Erreur lors des avis pour {nom_restaurant} : {e}")

driver.quit()
logging.info("Scraping terminé.")

2025-08-26 13:23:36,017 - INFO - Nouveau fichier Excel créé.
2025-08-26 13:23:46,412 - WARNING - Aucun bouton de consentement détecté
2025-08-26 13:25:19,455 - INFO - Avis sauvegardés pour Cafe Árabe (1/41)
2025-08-26 13:25:29,331 - WARNING - Aucun bouton de consentement détecté
2025-08-26 13:26:02,821 - INFO - Avis sauvegardés pour L'Mdina Marrakech (2/41)
2025-08-26 13:26:12,009 - WARNING - Aucun bouton de consentement détecté
2025-08-26 13:27:45,634 - INFO - Avis sauvegardés pour Naranj Libanese (3/41)
2025-08-26 13:27:55,387 - WARNING - Aucun bouton de consentement détecté
2025-08-26 13:29:28,358 - INFO - Avis sauvegardés pour La table du Palais (4/41)
2025-08-26 13:29:38,025 - WARNING - Aucun bouton de consentement détecté
2025-08-26 13:31:11,358 - INFO - Avis sauvegardés pour Ksar Essaoussan (5/41)
2025-08-26 13:31:20,754 - WARNING - Aucun bouton de consentement détecté
2025-08-26 13:32:54,058 - INFO - Avis sauvegardés pour L' Escapade Marrakech (6/41)
2025-08-26 13:33:03,658 - W

In [5]:
import re
import os
# === CONFIGURATION LOGGING ===
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("scraping-restaurants-marrakech-part_3.log", encoding="utf-8"),
        logging.StreamHandler()
    ]
)

# === CONFIGURATION SELENIUM ===
path = r"C:\Users\PC\Desktop\Scrapping-chromedriver\chromedriver.exe"
service = Service(executable_path=path)
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=service, options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})

wait = WebDriverWait(driver, 10)

# === Fichiers ===
input_file = "restaurants_links_marrakech_part3.xlsx"   # change selon la partie que tu veux scraper
output_file = "restaurants_reviews_marrakech3.xlsx"

# === Charger les liens ===
df_links = pd.read_excel(input_file)

# === Charger ou créer fichier des résultats ===
if os.path.exists(output_file):
    df = pd.read_excel(output_file)
    logging.info(f"Fichier existant trouvé avec {len(df)} lignes.")
else:
    df = pd.DataFrame(columns=["Nom du Restaurant", "Adresse", "Latitude", "Longitude", "Prix", "Nom", "Note", "Date", "Commentaire"])
    logging.info("Nouveau fichier Excel créé.")

# === Scraping ===
for i, row in df_links.iterrows():
    restaurant_url = row["Lien"]
    ville = row["Ville"]

    try:
        driver.get(restaurant_url)
        time.sleep(3)
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[.='Tout accepter']"))
            )
            accept_button.click()
            logging.info("Cookies acceptés")
            time.sleep(2)
        except:
            logging.warning("Aucun bouton de consentement détecté")
    except Exception as e:
        logging.error(f"Impossible de charger {restaurant_url} : {e}")
        continue

    # Récupération infos restaurant
    try:
        nom_restaurant = driver.find_element(By.XPATH, "//h1[contains(@class, 'DUwDvf')]").text
    except:
        nom_restaurant = "Nom inconnu"

    try:
        adresse = driver.find_element(By.XPATH, "//div[contains(@class, 'Io6YTe')]").text
    except:
        adresse = "Adresse inconnue"

    # Vérifier si déjà sauvegardé
    if ((df["Nom du Restaurant"] == nom_restaurant) & (df["Adresse"] == adresse)).any():
        logging.info(f"Déjà récupéré : {nom_restaurant} - {adresse}")
        continue

    try:
        current_url = driver.current_url
        match = re.search(r"/@([0-9\.\-]+),([0-9\.\-]+)", current_url)
        latitude, longitude = (match.group(1), match.group(2)) if match else ("", "")
    except:
        latitude, longitude = "", ""

    try:
        prix = driver.find_element(By.XPATH, "//span[contains(text(),'MAD') or contains(text(),'د.م') or contains(text(), '$')]").text
    except:
        prix = ""

    # === Ouvrir et scraper la section Avis ===
    all_reviews = []
    try:
        avis_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Avis')]")))
        avis_button.click()
        time.sleep(3)

        scrollable_div = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@jslog,'26354;mutable:true;')]")))

        # Scroll des avis
        start_time = time.time()
        previous_height = -1
        while True:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(2)
            current_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if current_height == previous_height or (time.time() - start_time) > 60:
                break
            previous_height = current_height

        review_ids_seen = set()
        avis_elements = driver.find_elements(By.XPATH, "//div[@data-review-id]")

        for avis in avis_elements:
            review_id = avis.get_attribute("data-review-id")
            if review_id in review_ids_seen:
                continue
            review_ids_seen.add(review_id)

            nom = avis.find_element(By.CLASS_NAME, "d4r55").text if avis.find_elements(By.CLASS_NAME, "d4r55") else ""
            note = avis.find_element(By.CLASS_NAME, "kvMYJc").get_attribute("aria-label") if avis.find_elements(By.CLASS_NAME, "kvMYJc") else ""
            date = avis.find_element(By.CLASS_NAME, "rsqaWe").text if avis.find_elements(By.CLASS_NAME, "rsqaWe") else ""
            commentaire = avis.find_element(By.CLASS_NAME, "wiI7pd").text if avis.find_elements(By.CLASS_NAME, "wiI7pd") else ""

            all_reviews.append([nom_restaurant, adresse, latitude, longitude, prix, nom, note, date, commentaire])

        # === Ajout au DataFrame et sauvegarde après le restaurant ===
        if all_reviews:
            df = pd.concat([df, pd.DataFrame(all_reviews, columns=df.columns)], ignore_index=True)
            df.to_excel(output_file, index=False)
            logging.info(f"Avis sauvegardés pour {nom_restaurant} ({i+1}/{len(df_links)})")
        else:
            logging.info(f"Aucun avis trouvé pour {nom_restaurant}")

    except Exception as e:
        logging.error(f"Erreur lors des avis pour {nom_restaurant} : {e}")

driver.quit()
logging.info("Scraping terminé.")

2025-08-26 14:30:25,775 - INFO - Nouveau fichier Excel créé.
2025-08-26 14:30:36,633 - WARNING - Aucun bouton de consentement détecté
2025-08-26 14:32:09,831 - INFO - Avis sauvegardés pour Restaurant le Meditérranée (1/40)
2025-08-26 14:32:20,176 - WARNING - Aucun bouton de consentement détecté
2025-08-26 14:33:53,318 - INFO - Avis sauvegardés pour Zeitoun Café Kasbah (2/40)
2025-08-26 14:34:02,689 - WARNING - Aucun bouton de consentement détecté
2025-08-26 14:35:36,247 - INFO - Avis sauvegardés pour Mazel مزال Cafe (3/40)
2025-08-26 14:35:45,860 - WARNING - Aucun bouton de consentement détecté
2025-08-26 14:37:19,486 - INFO - Avis sauvegardés pour Corner Cafe (4/40)
2025-08-26 14:37:29,364 - WARNING - Aucun bouton de consentement détecté
2025-08-26 14:39:03,131 - INFO - Avis sauvegardés pour L'Ô À LA BOUCHE - Marrakech (5/40)
2025-08-26 14:39:13,070 - WARNING - Aucun bouton de consentement détecté
2025-08-26 14:40:46,960 - INFO - Avis sauvegardés pour Le Bistro Arabe - Restaurant et j